In [50]:
#Imports
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#Data paths
datadir = os.path.join(os.getcwd(), 'archive')

data_map = {
"customers" : os.path.join(datadir, 'olist_customers_dataset.csv'),
"geolocation" : os.path.join(datadir, 'olist_geolocation_dataset.csv'),
#Orders
"order_items" : os.path.join(datadir, 'olist_order_items_dataset.csv'),
"order_payments" : os.path.join(datadir, 'olist_order_payments_dataset.csv'),
"order_reviews" : os.path.join(datadir, 'olist_order_reviews_dataset.csv'),
# MAIN DATA
"orders" : os.path.join(datadir, 'olist_orders_dataset.csv'),
#Products
"products" : os.path.join(datadir, 'olist_products_dataset.csv'),
"sellers" : os.path.join(datadir, 'olist_sellers_dataset.csv'),
"prod2cat_translation" : os.path.join(datadir, 'product_category_name_translation.csv')
}

def load(df_name):
    return pd.read_csv(data_map[df_name])

In [51]:
orderitems_df = load('order_items')
orderitems_df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [52]:
orders_main = load('orders')
print(orders_main.shape)
orders_main = orders_main.loc[orders_main.order_status == 'delivered']
print(orders_main.shape)
orders_main.head()

(99441, 8)
(96478, 8)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [53]:
is_late = orders_main.order_delivered_customer_date > orders_main.order_estimated_delivery_date
late_orders = orders_main.loc[is_late]

In [54]:
orders_main['is_late'] = False
orders_main.loc[is_late,'is_late'] = True
orders_main.loc[~is_late,'is_late'] = False
orders_main.is_late.value_counts()

False    88652
True      7826
Name: is_late, dtype: int64

In [55]:
orders_main['order_delivered_customer_date'] = pd.to_datetime(orders_main['order_delivered_customer_date'])
orders_main['order_delivered_carrier_date'] = pd.to_datetime(orders_main['order_delivered_carrier_date'])
orders_main['delivery_time'] = orders_main.order_delivered_customer_date - orders_main.order_delivered_carrier_date
orders_main['delivery_time'] = orders_main['delivery_time'].dt.days
print(orders_main.shape)
orders_main = orders_main.loc[~orders_main.delivery_time.isnull()] #Remove null rows
print(orders_main.shape)
orders_main = orders_main.loc[orders_main.delivery_time >= 0] #Remove invalid
print(orders_main.shape)

(96478, 10)
(96469, 10)
(96446, 10)


# Incorporate Product Info

In [56]:
prods = load('products')
prods

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [57]:
ord2prod = orderitems_df.merge(prods, on='product_id')
ord_stats = ord2prod.groupby(['order_id', 'product_id', 'seller_id'])[[ 'product_weight_g','price', 'freight_value',]].sum()
ord_stats

,,,product_weight_g,price,freight_value
order_id,product_id,seller_id,,,
00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,650.0,58.90,13.29
00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,30000.0,239.90,19.93
000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,3050.0,199.00,17.87
00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,200.0,12.99,12.79
00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,3750.0,199.90,18.14
...,...,...,...,...,...
fffc94f6ce00a00581880bf54a75a037,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,10150.0,299.99,43.41
fffcd46ef2263f404302a634eb57f7eb,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,8950.0,350.00,36.53
fffce4705a9662cd70adb13d4a31832d,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,967.0,99.90,16.95


## Find Distance between sellers and customers

In [58]:
geoloc = load('geolocation').set_index('geolocation_zip_code_prefix')
# geoloc['geolocation_lat'] = geoloc['geolocation_lat']
geoloc

,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
geolocation_zip_code_prefix,,,,
1037,-23.545621,-46.639292,sao paulo,SP
1046,-23.546081,-46.644820,sao paulo,SP
1046,-23.546129,-46.642951,sao paulo,SP
1041,-23.544392,-46.639499,sao paulo,SP
1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...
99950,-28.068639,-52.010705,tapejara,RS
99900,-27.877125,-52.224882,getulio vargas,RS
99950,-28.071855,-52.014716,tapejara,RS


from tqdm import tqdm
mapper = {}
for zipc in tqdm(geoloc.index[:]):
    row = geoloc.loc[zipc].head(n=1)
    # print(row)
    if 'geolocation_lat' not in row or 'geolocation_lng' not in row or isinstance(row['geolocation_lng'].values, np.ndarray):
        # print(f'Skipping {zipc}')
        continue
    print(type(row['geolocation_lng']))
    mapper[f'{zipc}_lat'] = row['geolocation_lng'].values[0]
    mapper[f'{zipc}_lng'] = row['geolocation_lng'].values[0]
mapper

def get_customer_lat(zip_code):
    lat = geoloc.loc[zip_code].head(n=1)['geolocation_lat']
    return lat.values[0] if isinstance(lat, np.ndarray) else lat

In [59]:
customers = load('customers')
#Filter out those who aren't in geo index
# customers = customers.loc[customers['customer_zip_code_prefix'].isin(geoloc.index)].head()
# print(customers)
# customers_merged = customers.merge(geoloc, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix')
# customers['lat'] = customers['customer_zip_code_prefix'].apply(get_customer_lat)
customers

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [60]:
sellers = load('sellers')
sellers

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS


In [61]:
ordstats_sellers = ord_stats.reset_index().merge(sellers, on='seller_id')
ordstats_sellers

,order_id,product_id,seller_id,product_weight_g,price,freight_value,seller_zip_code_prefix,seller_city,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,650.0,58.90,13.29,27277,volta redonda,SP
1,0188777fe321843a18be24a6e9aa1e53,436c8d57ff8d4aa254318e9bd9b48c83,48436dade18ac8b2bce089ec2a041202,530.0,55.90,9.94,27277,volta redonda,SP
2,0191bbb5166d4b9a68df11797d5acfa9,ec02a5d380128f7a188e9ce8f3ddd832,48436dade18ac8b2bce089ec2a041202,2275.0,132.90,20.17,27277,volta redonda,SP
3,03054d8a8eefc2981cfad06f58e27979,3e31fd1419cc4cd6adad1925e72e3a03,48436dade18ac8b2bce089ec2a041202,400.0,58.90,15.16,27277,volta redonda,SP
4,0c325ea6925de749e1420be0cf43587c,3a135477bf41350a0b1d8afdf616d446,48436dade18ac8b2bce089ec2a041202,600.0,139.90,16.74,27277,volta redonda,SP
...,...,...,...,...,...,...,...,...,...
102420,fdaf98feac227eb978d8f33f36e0231d,f739e9151702508b18f796c53005e5e9,d1aa1ec0839dcab73a6161130eb1f94a,1000.0,119.90,16.14,80220,curitiba,PR
102421,fdfeec75aa07c1d2a88b17b8fbfc7b8f,11059273f4bc1a293777e98c89807c62,dbc51f5e45d654ecc16cb68e6817ecea,1050.0,199.99,18.14,14402,franca,SP
102422,fe68b52db13993f58175fa589125d345,f4135cbdece8245560f7be179533797a,3fefda3299e6dfaea3466ef346a3571a,500.0,209.00,16.21,82620,curitiba,PR
102423,fefacc66af859508bf1a7934eab1e97f,69c590f7ffc7bf8db97190b6cb6ed62e,80ceebb4ee9b31afb6c6a916a574a1e2,5660.0,6729.00,193.21,86026,londrina,PR


In [62]:
orders_full = orders_main.merge(customers, on='customer_id').merge(ordstats_sellers, on = 'order_id')
print(orders_full[['order_id', 'customer_id', 'seller_id']].duplicated().any())
print(orders_full[['order_id', 'customer_id', 'seller_id', 'product_id']].duplicated().any()) #UNIQUE
orders_full = orders_full.set_index(['order_id', 'customer_id', 'seller_id', 'product_id'])
orders_full

True
False


,,,,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,is_late,delivery_time,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_weight_g,price,freight_value,seller_zip_code_prefix,seller_city,seller_state
order_id,customer_id,seller_id,product_id,,,,,,,,,,,,,,,,,,
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,3504c0cb71d7fa48d967e0e4c94d59d9,87285b34884572647811a353c7ac498a,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,False,6.0,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,500.0,29.99,8.72,9350,maua,SP
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,289cdb325fb7e7f891c38608bf9e0962,595fac2a385ac33a80bd5114aec74eb8,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,False,12.0,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,400.0,118.70,22.76,31570,belo horizonte,SP
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,4869f7a5dfa277a7dca6462dcf3b52b2,aa4383b373c6aca5d8797843e5594415,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,False,9.0,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,420.0,159.90,19.22,14840,guariba,SP
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,66922902710d126a0e7d26b0e3805106,d0b61bfb1de832b15ba9d266ca96e5b0,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,False,9.0,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,450.0,45.00,27.20,31842,belo horizonte,MG
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2c9e548be18521d1c43cde1c582c6de8,65266b2da20d04dbe00c5c2d3bb7859e,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,False,1.0,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,250.0,19.90,8.72,8752,mogi das cruzes,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,e24fc9fcd865784fb25705606fe3dfe7,ac35486adb7b02598c182c2ff2e05254,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,False,7.0,6359f309b166b0196dbf7ad2ac62bb5a,12209,sao jose dos campos,SP,1175.0,72.00,13.08,12913,braganca paulista,SP
63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,1f9ab4708f3056ede07124aad39a2554,f1d4ce8c6dd66c47bbaa8c6781c2a923,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,False,20.0,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP,4950.0,174.90,20.10,17602,tupa,SP
83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,d50d79cb34e38265a8649c383dcffd48,b80910977a37536adeddd63663f916ad,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,False,23.0,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA,13300.0,205.99,65.02,8290,sao paulo,SP


# Find kind of product

In [63]:
products_df = load('products')
translate = load("prod2cat_translation")
print(f'Products: {products_df.shape}')
print(f'Translate: {translate.shape}')
products_df.head()

prod_translated = products_df.merge(translate, on='product_category_name')
print(prod_translated.shape)
prod_translated.head()

Products: (32951, 9)
Translate: (71, 2)
(32328, 10)


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,6a2fb4dd53d2cdb88e0432f1284a004c,perfumaria,39.0,346.0,2.0,400.0,27.0,5.0,20.0,perfumery
2,0d009643171aee696f4733340bc2fdd0,perfumaria,52.0,150.0,1.0,422.0,21.0,16.0,18.0,perfumery
3,b1eae565a61935e0011ee7682fef9dc9,perfumaria,49.0,460.0,2.0,267.0,17.0,13.0,17.0,perfumery
4,8da90b37f0fb171b4877c124f965b1f6,perfumaria,56.0,733.0,3.0,377.0,18.0,13.0,15.0,perfumery


In [64]:
mapper = {}
for pid in prod_translated.index:
    row = prod_translated.loc[pid]
    # print(prod_translated.loc[pid]['product_category_name_english'])
    mapper[row['product_id']] = row['product_category_name_english']
# mapper

In [65]:
def get_catname(pid):
    if pid not in mapper:
        return 'unclassified'
    return mapper[pid]
orders_full.reset_index(inplace=True)
orders_full['product_category'] = orders_full.product_id.apply(get_catname)
orders_full.set_index(['order_id', 'customer_id', 'seller_id', 'product_id'], inplace=True)
orders_full

,,,,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,is_late,delivery_time,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_weight_g,price,freight_value,seller_zip_code_prefix,seller_city,seller_state,product_category
order_id,customer_id,seller_id,product_id,,,,,,,,,,,,,,,,,,,
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,3504c0cb71d7fa48d967e0e4c94d59d9,87285b34884572647811a353c7ac498a,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,False,6.0,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,500.0,29.99,8.72,9350,maua,SP,housewares
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,289cdb325fb7e7f891c38608bf9e0962,595fac2a385ac33a80bd5114aec74eb8,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,False,12.0,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,400.0,118.70,22.76,31570,belo horizonte,SP,perfumery
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,4869f7a5dfa277a7dca6462dcf3b52b2,aa4383b373c6aca5d8797843e5594415,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,False,9.0,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,420.0,159.90,19.22,14840,guariba,SP,auto
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,66922902710d126a0e7d26b0e3805106,d0b61bfb1de832b15ba9d266ca96e5b0,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,False,9.0,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,450.0,45.00,27.20,31842,belo horizonte,MG,pet_shop
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2c9e548be18521d1c43cde1c582c6de8,65266b2da20d04dbe00c5c2d3bb7859e,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,False,1.0,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,250.0,19.90,8.72,8752,mogi das cruzes,SP,stationery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,e24fc9fcd865784fb25705606fe3dfe7,ac35486adb7b02598c182c2ff2e05254,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,False,7.0,6359f309b166b0196dbf7ad2ac62bb5a,12209,sao jose dos campos,SP,1175.0,72.00,13.08,12913,braganca paulista,SP,health_beauty
63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,1f9ab4708f3056ede07124aad39a2554,f1d4ce8c6dd66c47bbaa8c6781c2a923,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,False,20.0,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP,4950.0,174.90,20.10,17602,tupa,SP,baby
83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,d50d79cb34e38265a8649c383dcffd48,b80910977a37536adeddd63663f916ad,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,False,23.0,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA,13300.0,205.99,65.02,8290,sao paulo,SP,home_appliances_2


## Model Fitting

In [66]:
# data = orders_full[['product_weight_g', 'customer_state', 'seller_state', 'is_late']]

features = ['product_weight_g', 'customer_state', 'seller_state', 'product_category']
data = orders_full[features + ['delivery_time']]

# features = ['product_category']
# data = orders_full[features + ['product_weight_g']]
# features = ['product_weight_g']
X = data[features]
y = data['delivery_time']
# y = data['product_weight_g']

#Encode cat
X = pd.get_dummies(X).values
y = y.values
#Downsample positive class

print(X)
print(y)
print(X.shape)
print(y.shape)

[[  500.     0.     0. ...     0.     0.     0.]
 [  400.     0.     0. ...     0.     0.     0.]
 [  420.     0.     0. ...     0.     0.     0.]
 ...
 [13300.     0.     0. ...     0.     0.     0.]
 [13100.     0.     0. ...     0.     0.     0.]
 [  150.     0.     0. ...     0.     0.     0.]]
[ 6. 12.  9. ... 23. 13.  6.]
(100144, 122)
(100144,)


data.delivery_time.plot.hist(bins=100)
plt.show()

In [67]:
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder()
# encoder.fit(X)
# X = X.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)
print(f'X_train shape: {X_train.shape} \t X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape} \t y_test shape: {y_test.shape}')

X_train shape: (95136, 122) 	 X_test shape: (5008, 122)
y_train shape: (95136,) 	 y_test shape: (5008,)


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, classification_report


# model = LogisticRegression(random_state=0, n_jobs=-1).fit(X_train, y_train)
model = RandomForestClassifier(n_jobs=-1)
model.fit(X_train, y_train)
yhat_train = model.predict(X_train)
yhat_test = model.predict(X_test)

accs = {}

accs['train_acc'] = accuracy_score(y_train, yhat_train)
accs['test_acc'] = accuracy_score(y_test, yhat_test)
accs['auc_train'] = roc_auc_score(y_train, yhat_train)
accs['auc_test'] = roc_auc_score(y_test, yhat_test)
accs['f1_test'] = f1_score(y_test, yhat_test)
print(accs)
print(classification_report(y_test, yhat_test))

In [68]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn import linear_model
from sklearn.svm import SVR

# model = RandomForestRegressor(n_estimators = 100 , max_depth=5, random_state=0, n_jobs=-1)
# model = MLPRegressor(random_state=1, max_iter=500)
model = linear_model.Ridge(alpha=10.0)
# model = linear_model.Lasso()
# model = SVR()
# model = linear_model.HuberRegressor()


model.fit(X_train, y_train)

yhat_train = model.predict(X_train)
yhat_test = model.predict(X_test)

#Errors
errors = {}
errors['train_rmse'] = np.sqrt(mean_squared_error(y_train, yhat_train))
errors['test_rmse'] = np.sqrt(mean_squared_error(y_test, yhat_test))
errors['train_mae'] = mean_absolute_error(y_train, yhat_train)
errors['test_mae'] = mean_absolute_error(y_test, yhat_test)
errors['train_r2'] = r2_score(y_train, yhat_train)
errors['test_r2'] = r2_score(y_test, yhat_test)

errors

{'train_rmse': 7.76614526217454,
 'test_rmse': 7.795914433548341,
 'train_mae': 4.6991444712777,
 'test_mae': 4.69244718307861,
 'train_r2': 0.207779044411734,
 'test_r2': 0.20340120817538432}

In [69]:
np.percentile(y, 75)

11.0

In [70]:
pd.get_dummies(data.loc[data.product_weight_g > 1000][features]).head(n=1)

,,,,product_weight_g,customer_state_AC,customer_state_AL,customer_state_AM,customer_state_AP,customer_state_BA,customer_state_CE,customer_state_DF,customer_state_ES,customer_state_GO,...,product_category_signaling_and_security,product_category_small_appliances,product_category_small_appliances_home_oven_and_coffee,product_category_sports_leisure,product_category_stationery,product_category_tablets_printing_image,product_category_telephony,product_category_toys,product_category_unclassified,product_category_watches_gifts
order_id,customer_id,seller_id,product_id,,,,,,,,,,,,,,,,,,,,,
a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,8581055ce74af1daba164fdbd55a40de,060cb19345d90064d1015407193c233d,7150.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
data

,,,,product_weight_g,customer_state,seller_state,product_category,delivery_time
order_id,customer_id,seller_id,product_id,,,,,
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,3504c0cb71d7fa48d967e0e4c94d59d9,87285b34884572647811a353c7ac498a,500.0,SP,SP,housewares,6.0
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,289cdb325fb7e7f891c38608bf9e0962,595fac2a385ac33a80bd5114aec74eb8,400.0,BA,SP,perfumery,12.0
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,4869f7a5dfa277a7dca6462dcf3b52b2,aa4383b373c6aca5d8797843e5594415,420.0,GO,SP,auto,9.0
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,66922902710d126a0e7d26b0e3805106,d0b61bfb1de832b15ba9d266ca96e5b0,450.0,RN,MG,pet_shop,9.0
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2c9e548be18521d1c43cde1c582c6de8,65266b2da20d04dbe00c5c2d3bb7859e,250.0,SP,SP,stationery,1.0
...,...,...,...,...,...,...,...,...
9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,e24fc9fcd865784fb25705606fe3dfe7,ac35486adb7b02598c182c2ff2e05254,1175.0,SP,SP,health_beauty,7.0
63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,1f9ab4708f3056ede07124aad39a2554,f1d4ce8c6dd66c47bbaa8c6781c2a923,4950.0,SP,SP,baby,20.0
83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,d50d79cb34e38265a8649c383dcffd48,b80910977a37536adeddd63663f916ad,13300.0,BA,SP,home_appliances_2,23.0
